# Deep Learning in Practice

MVA 2021-2022

Elias Masquil: eliasmasquil@gmail.com

Nicolas Violante: nviolante96@gmail.com

## Visualization of CNN: Grad-CAM
* **Objective**: Convolutional Neural Networks are widely used on computer vision. It is powerful for processing grid-like data. However we hardly know how and why it works, due to the lack of decomposability into individually intuitive components. In this assignment, we use Grad-CAM, which highlights the regions of the input image that were important for the neural network prediction.

* **To be submitted within 2 weeks**: this notebook, **cleaned** (i.e. without results, for file size reasons: `menu > kernel > restart and clean`), in a state ready to be executed (if one just presses 'Enter' till the end, one should obtain all the results for all images) with a few comments at the end. No additional report, just the notebook!

* NB: if `PIL` is not installed, try `conda install pillow`.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
import matplotlib.pyplot as plt
import pickle
import urllib.request

import numpy as np
from PIL import Image

%matplotlib inline

### Download the Model
We provide you a pretrained model `ResNet-34` for `ImageNet` classification dataset.
* **ImageNet**: A large dataset of photographs with 1 000 classes.
* **ResNet-34**: A deep architecture for image classification.

In [ ]:
resnet34 = models.resnet34(pretrained=True)
resnet34.eval() # set the model to evaluation mode

![ResNet34](https://miro.medium.com/max/1050/1*Y-u7dH4WC-dXyn9jOG4w0w.png)

In [ ]:
classes = pickle.load(urllib.request.urlopen('https://gist.githubusercontent.com/yrevar/6135f1bd8dcf2e0cc683/raw/d133d61a09d7e5a3b36b8c111a8dd5c4b5d560ee/imagenet1000_clsid_to_human.pkl') )

### Input Images
We provide you 20 images from ImageNet (download link on the webpage of the course or download directly using the following command line,).<br>
In order to use the pretrained model resnet34, the input image should be normalized using `mean = [0.485, 0.456, 0.406]`, and `std = [0.229, 0.224, 0.225]`, and be resized as `(224, 224)`.

In [ ]:
def preprocess_image(dir_path):
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

    dataset = datasets.ImageFolder(dir_path, transforms.Compose([
            transforms.Resize(256), 
            transforms.CenterCrop(224), # resize the image to 224x224
            transforms.ToTensor(), # convert numpy.array to tensor
            normalize])) #normalize the tensor

    return (dataset)

In [ ]:
# The images should be in a *sub*-folder of "data/" (ex: data/TP2_images/images.jpg) and *not* directly in "data/"!
# otherwise the function won't find them

import os
os.mkdir("data")
os.mkdir("data/TP2_images")
!cd data/TP2_images && wget "https://www.lri.fr/~gcharpia/deeppractice/2022/TP2/TP2_images.zip" && unzip TP2_images.zip
dir_path = "data/" 
dataset = preprocess_image(dir_path)

In [ ]:
# show the orignal image 
index = 5
input_image = Image.open(dataset.imgs[index][0]).convert('RGB')
plt.imshow(input_image)

In [ ]:
output = resnet34(dataset[index][0].view(1, 3, 224, 224))
values, indices = torch.topk(output, 3)
print("Top 3-classes:", indices[0].numpy(), [classes[x] for x in indices[0].numpy()])
print("Raw class scores:", values[0].detach().numpy())

### Grad-CAM 
* **Overview:** Given an image, and a category (â€˜tiger catâ€™) as input, we forward-propagate the image through the model to obtain the `raw class scores` before softmax. The gradients are set to zero for all classes except the desired class (tiger cat), which is set to 1. This signal is then backpropagated to the `rectified convolutional feature map` of interest, where we can compute the coarse Grad-CAM localization (blue heatmap).


* **To Do**: Define your own function Grad_CAM to achieve the visualization of the given images. For each image, choose the top-3 possible labels as the desired classes. Compare the heatmaps of the three classes, and conclude. 


* **Hints**: 
 + We need to record the output and grad_output of the feature maps to achieve Grad-CAM. In pytorch, the function `Hook` is defined for this purpose. Read the tutorial of [hook](https://pytorch.org/tutorials/beginner/former_torchies/nnft_tutorial.html#forward-and-backward-function-hooks) carefully. 
 + The pretrained model resnet34 doesn't have an activation function after its last layer, the output is indeed the `raw class scores`, you can use them directly. 
 + The size of feature maps is 7x7, so your heatmap will have the same size. You need to project the heatmap to the resized image (224x224, not the original one, before the normalization) to have a better observation. The function [`torch.nn.functional.interpolate`](https://pytorch.org/docs/stable/nn.functional.html?highlight=interpolate#torch.nn.functional.interpolate) may help.  
 + Here is the link of the paper [Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization](https://arxiv.org/pdf/1610.02391.pdf)

![Grad-CAM](https://da2so.github.io/assets/post_img/2020-08-10-GradCAM/2.png)

#### Implementation

In [ ]:
from functools import partial
import random

class GradCAM():
    def __init__(self, model):
        # Trained model
        self.model = model
        # Internal variables for storing activations and gradients with hooks
        self.activations = []
        self.gradients = []
        # Registering hooks to retain useful values
        self.model.layer4[-1].conv2.register_forward_hook(self.save_activations)
        self.model.layer4[-1].conv2.register_full_backward_hook(self.save_gradients)

    def save_activations(self, model, layer_input, layer_output):
        """
        Save the k activations maps A^k
        """
        self.activations = layer_output.data


    def save_gradients(self, model, grad_input, grad_output):
        """
        Save the k gradients dy^c/dA^k
        """
        self.gradients = grad_output[0]


    def compute_top_3(self, image):
        """
        Computes the 3 GradCAMs associated to the top 3 possible classes
        """
        # Forward pass
        output = self.model(image)
        # Select top 3 classes
        _, indices = torch.topk(output, 3)
        # Save GradCAMs
        heatmaps = []
        predicted_labels = []
        for top_k in range(3):
            idx = indices[0][top_k]
            # Computes the gradient of the prediction of the class of interest c
            output[0, idx].backward(retain_graph=True)
            # GradCAM weights computed by average pooling
            weights = torch.nn.AvgPool2d(self.gradients.shape[-2:])(self.gradients)
            # GradCAM computation by averaging activations + ReLU
            heatmap = torch.relu(torch.sum(self.activations * weights, 1))
            # Upscaling heatmap
            heatmap = nn.functional.interpolate(heatmap[None, ...], image.shape[-2:], mode="bilinear")
            heatmaps.append(heatmap[0,0,...])
            predicted_labels.append(idx.item())
        return heatmaps, predicted_labels

In [ ]:
resnet34 = models.resnet34(pretrained=True)
resnet34.eval() # set the model to evaluation mode
grad_cam = GradCAM(resnet34)

#### Visualization and comments

In [ ]:
for image_index in range(len(dataset)):
    heatmaps, predicted_labels = grad_cam.compute_top_3(dataset[image_index][0].view(1, 3, 224, 224))
    img = Image.open(dataset.imgs[image_index][0]).convert('RGB')
    img = img.resize((224,224))
    plt.figure(figsize=(15,5))
    for i in range(len(heatmaps)):
        plt.subplot(1,3,+i+1)
        plt.imshow(img)
        plt.title(classes[predicted_labels[i]])
        plt.imshow(heatmaps[i], alpha=0.5)

In the majority of the images (around 14) the activation maps for the top-3 predicted classes are really similar. This seems to indicate, that in those cases, the top-3 classes are also close, then the model focus on similar parts of the image. (E.g the elephant)

However, there are cases where the activation maps concentrate at different parts of the image. For example in the 4th image (the dog one), for predicting the second class, the model pays attention to a different part of the body of the dog.

In general, we see that the activation maps generated with this method are able to localize the category in the image. Nevertheless, there are two cases where some of the maps are not concentrated around the class instance (when the model predicts Chesapeake Bay Retriever and Otter). This might indicate that the model is learning some pattern for predicting that class not directly related with the instance (e.g the people in background in the Chesapeake Bay Retriever example).